In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
# vous devez importez les données ici

data = pd.read_csv('/content/gdrive/MyDrive/Projet/legal_text_classification.csv')

In [4]:
data.head(5)

,case_id,case_outcome,case_title,case_text
0,Case1,cited,Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Lt...,Ordinarily that discretion will be exercised s...
1,Case2,cited,Black v Lipovac [1998] FCA 699 ; (1998) 217 AL...,The general principles governing the exercise ...
2,Case3,cited,Colgate Palmolive Co v Cussons Pty Ltd (1993) ...,Ordinarily that discretion will be exercised s...
3,Case4,cited,Dais Studio Pty Ltd v Bullett Creative Pty Ltd...,The general principles governing the exercise ...
4,Case5,cited,Dr Martens Australia Pty Ltd v Figgins Holding...,The preceding general principles inform the ex...


In [5]:
data.shape

(24985, 4)

In [6]:
data.describe(include=['O'])

,case_id,case_outcome,case_title,case_text
count,24985,24985,24985,24809
unique,24985,10,18581,17920
top,Case1,cited,Minister for Immigration and Ethnic Affairs v ...,submitted that this Court should hold that the...
freq,1,12219,70,42


In [7]:
data.isnull().sum()

case_id           0
case_outcome      0
case_title        0
case_text       176
dtype: int64

In [8]:
data['case_outcome'].unique()

array(['cited', 'applied', 'followed', 'referred to', 'related',
       'considered', 'discussed', 'distinguished', 'affirmed', 'approved'],
      dtype=object)

In [8]:
data['label'] = [0 if label == "cited" else 1
                                 if label=='applied'  else 2
                                 if label=='followed' else  3
                                 if label == "referred to" else 4
                                 if label=='related'  else 5
                                 if label=='considered' else 6
                                 if label == "discussed" else 7
                                 if label=='distinguished' else 8
                                 if label=='affirmed' else
                               9  for label in data['case_outcome']]

In [10]:
data['label'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
data['case_outcome']

0                cited
1                cited
2                cited
3                cited
4                cited
             ...      
24980            cited
24981            cited
24982            cited
24983    distinguished
24984    distinguished
Name: case_outcome, Length: 24985, dtype: object

In [9]:
data['text'] = [str(x) for x in data['case_text']]

In [13]:
data.isnull().sum()

case_id           0
case_outcome      0
case_title        0
case_text       176
label             0
text              0
dtype: int64

In [10]:
data = data.drop(columns=['case_id', 'case_outcome', 'case_title', 'case_text'])

In [15]:
data.shape

(24985, 2)

In [16]:
data.head(10)

,label,text
0,0,Ordinarily that discretion will be exercised s...
1,0,The general principles governing the exercise ...
2,0,Ordinarily that discretion will be exercised s...
3,0,The general principles governing the exercise ...
4,0,The preceding general principles inform the ex...
5,0,I accept that the making of a rolled up offer ...
6,0,The preceding general principles inform the ex...
7,0,On the question of the level of unreasonablene...
8,1,recent decision of the High Court in Australia...
9,2,Hexal Australia Pty Ltd v Roche Therapeutics I...


In [11]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.4 MB/s eta 0:00:00


In [12]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=6870e08726742b13a7a0a6b56344c45bc535c1d5743d94762ed8ad76bf6cb3e5
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [13]:
!pip install huggingface_hub

In [25]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertForSequenceClassification
from transformers.optimization import AdamW
from transformers import BertPreTrainedModel
from transformers import AutoConfig
from huggingface_hub import PyTorchModelHubMixin
import wandb

import pandas as pd
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from sklearn.model_selection import train_test_split

#import wandb  # monitoring

config = {
    "model_name": "bert-base-uncased",
    "max_length": 80,
    "hidden_state": 768,
    "data": data,
    "batch_size": 2,
    "learing_rate": 2e-5,
    "n_epochs": 1,
    "n_classes": 10,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}


tokenizer = BertTokenizer.from_pretrained(config['model_name'])

class MyDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.df = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        text = self.df["text"][index]
        label = self.df["label"][index]

        inputs = self.tokenizer(
            text=text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
           "label": torch.tensor(label),
        }


def dataloader(dataset, batch_size, shuffle):
    return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)




In [26]:
class CustomModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, model_name, n_classes):
        super(CustomModel, self).__init__()
        self.pretrained_model = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=10
        )  # hidden_state 786 Bert_base




    def forward(self, input_ids, attention_mask):

        output = self.pretrained_model(
            input_ids=input_ids, attention_mask=attention_mask
        )  # (batch, 768)

        # Utilisez la sortie "logits" du modèle pré-entraîné
        logits = output.logits

        # Réorganisez les dimensions des logits avant la classification linéaire
        #logits = logits.view(-1, self.pretrained_model.config.hidden_size)

        # Passez les logits par le classificateur linéaire
        #output = self.classifier(logits)


        return logits



In [27]:

def train_step(model, train_loader, optimizer, loss_fn, device):
    model.train()

    total_loss = 0

    for data in tqdm(train_loader, total=len(train_loader)):

        input_ids = data["input_ids"].squeeze(1).to(device)
        attention_mask = data["attention_mask"].to(device)
        label = data["label"].to(device)

        optimizer.zero_grad()

        output = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(
            output, label
        )
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)


def validation_step(model, validation_loader, loss_fn, device):

    total_loss = 0
    correct_prediction = 0

    with torch.no_grad():
        for data in tqdm(validation_loader, total=len(validation_loader)):
            input_ids = data["input_ids"].squeeze(1).to(device)
            attention_mask = data["attention_mask"].to(device)
            label = data["label"].to(device)

            output = model(input_ids=input_ids, attention_mask=attention_mask)

            loss = loss_fn(
                output, label
            )
            total_loss += loss.item()

            pred = torch.argmax(torch.softmax(output, dim=1), dim=1)
            correct_prediction += torch.sum(pred == label)

    return total_loss / len(validation_loader), 100 * correct_prediction / len(
        validation_loader
    )





In [34]:
wandb.login(key='yourkey')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:

wandb.init(project="bert_classification")

wandb: Currently logged in as: tatchum-ulrich. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230918_091857-1k8g03nr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run magic-wildflower-1
wandb: ⭐️ View project at https://wandb.ai/tatchum-ulrich/bert_classification
wandb: 🚀 View run at https://wandb.ai/tatchum-ulrich/bert_classification/runs/1k8g03nr


In [36]:



dataset =MyDataset(data=config['data'], tokenizer=tokenizer, max_length=config['max_length'] )

train_dataset , validation_dataset = train_test_split(dataset , test_size =0.2)

train_loader = dataloader(train_dataset , batch_size=config['batch_size'], shuffle=True)
validation_loader = dataloader(validation_dataset, batch_size=config['batch_size'] , shuffle=False)


model  = CustomModel(model_name=config['model_name'] , n_classes=10)
model.to(config['device'])

loss_fn = nn.CrossEntropyLoss()

optimizer = AdamW(model.parameters() , lr =config['learing_rate'])

for epoch in range(config['n_epochs']):

    loss_train =train_step(model , train_loader , optimizer , loss_fn ,config['device'])
    loss_validation , accuracy = validation_step(model , validation_loader , loss_fn , config['device'])

    wandb.log({'loss_train': loss_train,
          'loss_validation': loss_validation,
           'accuracy': accuracy
            })





    # sauvegarder
torch.save(model , 'bert-model_10.pth')

torch.save(tokenizer , 'bert-model.pth')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 2499/2499 [00:40<00:00, 61.47it/s]


In [38]:
from huggingface_hub import notebook_login
notebook_login()

In [39]:
model.push_to_hub('hot_bert_classification')
tokenizer.push_to_hub('hot_bert_classification')

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ulrichING/hot_bert_classification/commit/8c7dd09ac9f6d7bb32253b351aacac4fe41a3afe', commit_message='Upload tokenizer', commit_description='', oid='8c7dd09ac9f6d7bb32253b351aacac4fe41a3afe', pr_url=None, pr_revision=None, pr_num=None)